# 6장 트위터 데이터 수집하기

In [35]:
import oauth2
import json
import datetime
import time
from config import *

def oauth2_request(consumer_key, consumer_secret, access_token, access_secret):
    try:
        consumer = oauth2.Consumer(key=consumer_key, secret=consumer_secret)
        token = oauth2.Token(key=access_token, secret=access_secret)
        client = oauth2.Client(consumer, token)
        return client
    except Exception as e:
        print(e)
        return None
    
    
def get_user_timeline(client, screen_name, count=50, include_rts='False'):
    base = "https://api.twitter.com/1.1"
    node = "/statuses/user_timeline.json"
    fields = "?screen_name=%s&count=%s&include_rts=%s" % (screen_name, count, include_rts)
    #fields = "?screen_name=%s" % (screen_name)
    url = base + node + fields
    
    response, data = client.request(url)
    
    try:
        if response['status'] == '200':
            return json.loads(data.decode('utf-8'))
        else:
            print(response)
            print(response['status'])
    except Exception as e:
        print(e)
        return None
    
    
def getTwitterTwit(tweet, jsonResult):
    tweet_id = tweet['id_str']
    tweet_message = '' if 'text' not in tweet.keys() else tweet['text']
    screen_name = '' if 'user' not in tweent.keys() else tweet['user']['screen_name']
    tweet_link = ''
    
    if tweet['entities']['urls']:
        for i, val in enumerate(tweet['entitles']['urls']):
            tweet_link = tweet_link + tweet['entities']['urls'][i]['url'] + ''
    else:
        tweet_link = ''
    
    hashtags = ''
    if tweet['entities']['hashtags']:
        for i, val in enumerate(tweet['entities']['hashtags']):
            hashtags = hasgtags + tweet['entities']['hashtags'][i]['text'] + ''
    else:
        hashtags = ''
        
    if 'created_at' in tweet.keys():
        tweet_published = datetime.datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S +0000 %Y)')
        tweet_published = tweet_published + datetime.timedelta(hours=+9)
        tweet_published = tweet_published.strftime("%Y-%m-%d %H:%M:%S")
    else:
        tweet_published = ''
        
    num_favorite_count = 0 if 'favorite_count' not in tweet.keys() else tweet['favorite_count']
    num_comments = 0
    num_shares = 0 if 'retweet_count' not in tweet.keys() else tweet['retweet_count']
    num_likes = num_favorite_count
    num_loves = num_wows = num_hahas = num_sads = num_angrys = 0
    
    jsonResult.append({'post_id':tweet_id,
                      'message':tweet_message,
                      'name':screen_name,
                      'link':tweet_link,
                      'created_time':tweet_published,
                      'num_reactions':num_favorite_count,
                      'num_comments':num_comments,
                      'num_shares':num_shares,
                      'num_likes':num_likes,
                      'num_loves':num_loves,
                      'num_wows':num_wows,
                      'num_hahas':num_hahas,
                      'num_sads':num_sads,
                      'num_angrys':num_angrys,
                      'hashtags':hashtags})
    
    
def main():
    screen_name = 'T1_LOL'
    num_posts = 50
    jsonResult = []
    
    CONSUMER_KEY = 'ZIzHfI8x13sTyyvestefxDXU8'
    CONSUMER_SECRET = 'exOG5JS9myqdeZQ3fU2GA2mFOfuu3sGQOQuXZpaYp133D3NynR'
    ACCESS_TOKEN = '1246285995357552640-PNs1cCbgYNI4WRVtC1zkBghZuavrkU0'
    ACCESS_SECRET = 'ePOQQvsitkfhwKpRVN1HjcMhM3KaH9Kri3uzzivLbgeJv'
    
    client = oauth2_request(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET)
    tweets = get_user_timeline(client, screen_name)
    
    for tweet in tweets:
        getTwitterTwit(tweet, jsonResult)
        
    with open('%s_twitter.json' % (screen_name), 'w', encoding='utf-8') as outfile:
        str_ = json.dumps(jsonResult,
                         indent=4,
                         sort_keys=True,
                         ensure_ascii=False)
        print('%s_twitter.json SAVED' % (screen_name))
        
    
if __name__ == '__main__':
    main()
    

{'cache-control': 'no-cache, no-store, max-age=0', 'content-length': '62', 'content-type': 'application/json; charset=utf-8', 'date': 'Thu, 06 Aug 2020 15:24:28 GMT', 'server': 'tsa_m', 'set-cookie': 'personalization_id="v1_HW6+h5rHgEYV4icgD4nAyA=="; Max-Age=63072000; Expires=Sat, 6 Aug 2022 15:24:28 GMT; Path=/; Domain=.twitter.com; Secure; SameSite=None, guest_id=v1%3A159672746823789998; Max-Age=63072000; Expires=Sat, 6 Aug 2022 15:24:28 GMT; Path=/; Domain=.twitter.com; Secure; SameSite=None', 'strict-transport-security': 'max-age=631138519', 'x-connection-hash': '6f4d0c27d9841415dc53b059852966e4', 'x-response-time': '103', 'status': '400', '-content-encoding': 'gzip'}
400


TypeError: 'NoneType' object is not iterable

# 7장 네이버 데이터 수집하기

## 7.1 검색 API의 활용

In [19]:
# 비로그인 방식을 통한 네이버 뉴스, 블로그, 까페 글 검색

import os
import sys
import urllib.request
import datetime
import time
import json
from config import *

def get_request_url(url):
    
    # urllib.request.Request는 class(abstraction of a URL request)
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", "iqwsSNAxP1UnkVXEd75A")
    req.add_header("X-Naver-Client-Secret", "eOK_0IHNOx")
    
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success for URL: %s" % (datetime.datetime.now(), url))
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL: %s" % (datetime.datetime.now(), url))
        return None
    
def getNaverSearchResult(sNode, search_text, page_start, display):
    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % sNode
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(search_text), page_start, display)
    url = base + node + parameters
    
    retData = get_request_url(url)
    
    if (retData == None):
        return None
    else:
        return json.loads(retData)
    
def getPostData(post, jsonResult):
    
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    
    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    
    jsonResult.append({'title':title, 
                       'description':description,
                       'org_link':org_link, 
                       'link':org_link,
                       'pDate':pDate})
    return
    
def main():
    
    jsonResult = []
    
    sNode = 'news' #'news', 'blog', 'cafearticle'
    search_text = 'T1'
    display_count = 100
    
    jsonSearch = getNaverSearchResult(sNode, search_text, 1, display_count)
    
    while((jsonSearch != None) and (jsonSearch['display'] != 0)):
        for post in jsonSearch['items']:
            getPostData(post, jsonResult)
            
        nStart = jsonSearch['start'] + jsonSearch['display']
        jsonSearch = getNaverSearchResult(sNode, search_text, nStart, display_count)
        
    with open('%s_naver_%s.json' % (search_text, sNode), 'w', encoding='utf-8') as outfile:
        retJson = json.dumps(jsonResult,
                            indent=4,
                            sort_keys=True,
                            ensure_ascii=False)
        outfile.write(retJson)
        
    print('%s_naver_%s.json SAVED' % (search_text, sNode))
    
if __name__ == '__main__':
    main()

[2020-08-06 07:59:37.216871] Url Request Success for URL: https://openapi.naver.com/v1/search/news.json?query=T1&start=1&display=100
[2020-08-06 07:59:37.421332] Url Request Success for URL: https://openapi.naver.com/v1/search/news.json?query=T1&start=101&display=100
[2020-08-06 07:59:37.618633] Url Request Success for URL: https://openapi.naver.com/v1/search/news.json?query=T1&start=201&display=100
[2020-08-06 07:59:37.839750] Url Request Success for URL: https://openapi.naver.com/v1/search/news.json?query=T1&start=301&display=100
[2020-08-06 07:59:38.090727] Url Request Success for URL: https://openapi.naver.com/v1/search/news.json?query=T1&start=401&display=100
[2020-08-06 07:59:38.322713] Url Request Success for URL: https://openapi.naver.com/v1/search/news.json?query=T1&start=501&display=100
[2020-08-06 07:59:38.538778] Url Request Success for URL: https://openapi.naver.com/v1/search/news.json?query=T1&start=601&display=100
[2020-08-06 07:59:38.748003] Url Request Success for URL:

## 7.2 지도 API의 활용 - 카카오

(네이버는 서비스 중단)

In [25]:
import os
import sys
import urllib.request
import datetime
import time
import json
from config import *

native_app_key = '1d9651c42f3d6f438e7afdfe7d9a1ff0'
rest_api_key = '0d58fd941b80009d0ecb936b3787587a'
javascript_key = '6a5981bdfce0fbdb71c6b49b4595fdaa'
admin_key = '51838b22a226337ddea0b0285ddc202c'


def get_request_url(url):
    
    req = urllib.request.Request(url)
    req.add_header("Authorization", "KakaoAK "+rest_api_key)
    
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success for URL: %s" % (datetime.datetime.now(), url))
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL: %s" % (datetime.datetime.now(), url))
        return None
    

def getGeoData(address):
    
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(address)
    
    retData = get_request_url(url)
    
    if retData == None:
        return None
    else:
        return json.loads(retData)
    
    
def main():
    
    searching = 'T1'
    jsonResult = getGeoData(searching)
    
    if 'meta' in jsonResult.keys():
        print('총 검색 결과: ', jsonResult['meta']['total_count'])
        print('검색어: ', jsonResult['meta']['same_name']['keyword'])
        
        for item in jsonResult['documents']:
            print('===========================')
            print('주소: ', item['road_address_name'])
            print('위도: ', item['y'])
            print('경도: ', item['x'])
            
            
if __name__ == '__main__':
    main()

[2020-08-06 09:40:00.258526] Url Request Success for URL: https://dapi.kakao.com/v2/local/search/keyword.json?query=T1
총 검색 결과:  48
검색어:  t1
주소:  서울 강남구 선릉로 627
위도:  37.5124177833897
경도:  127.042836395143
주소:  인천 중구 공항로 271
위도:  37.4463799212184
경도:  126.456718819439
주소:  인천 중구 공항로 271
위도:  37.4480932459114
경도:  126.457027378139
주소:  인천 중구 공항로 271
위도:  37.4447868498249
경도:  126.453570715701
주소:  서울 광진구 광나루로56길 85
위도:  37.5357010612649
경도:  127.095764656407
주소:  인천 중구 공항로 271
위도:  37.4467374233447
경도:  126.455305893658
주소:  인천 중구 공항로 271
위도:  37.4452621939524
경도:  126.452708414339
주소:  충남 서산시 안견로 254-5
위도:  36.7780529696401
경도:  126.454997829694
주소:  인천 중구 공항로 271
위도:  37.4442803040782
경도:  126.457050413717
주소:  인천 중구 공항로 271
위도:  37.448056278615
경도:  126.451557959658
주소:  인천 중구 공항로 271
위도:  37.4417364131733
경도:  126.452480973879
주소:  인천 중구 공항로 272
위도:  37.4439789566915
경도:  126.452663519696
주소:  인천 중구 공항로 271
위도:  37.4454656563107
경도:  126.453841513812
주소:  경기 포천시 영북면 산정호수로322번길 20
위도:

# 9장 일반적인 웹 서비스 데이터 수집하기

## 9.1 이상한 나라의 앨리스의 맛있는 스프: BeautifulSoup

In [41]:
import urllib.request
from bs4 import BeautifulSoup
html = urllib.request.urlopen('https://movie.naver.com/movie/sdb/rank/rmovie.nhn')
soup = BeautifulSoup(html, 'html.parser')

tags = soup.findAll('div', attrs={'class':'tit3'})
for tag in tags:
    print(tag.a.text)

강철비2: 정상회담
다만 악에서 구하소서
반도
#살아있다
존 윅
소년시절의 너
알라딘
오케이 마담
세인트 주디
1942: 언노운 배틀
빅샤크3: 젤리몬스터 대소동
밤쉘: 세상을 바꾼 폭탄선언
강철비
온워드: 단 하루의 기적
블랙아웃 : 인베이젼 어스
프리즈너
키싱 부스 2
테넷
봉오동 전투
정도
애니멀 크래커
팬데믹
국제수사
승리호
고스트 오브 워
위대한 쇼맨
마티아스와 막심
팡파레
반교: 디텐션
블루 아워
카오산 탱고
기기괴괴 성형수
결백
큐리오사
인베이젼 2020
양자물리학
올드 가드
모든 것을 벗어던진 특별한 여행
파리의 인어
소년 아메드
극장판 짱구는 못말려: 신혼여행 허리케인~ 사라진 아빠!
키싱 부스
소셜노마드
워터 릴리스
가버나움
디바
기담
사라진 시간
킹스맨: 퍼스트 에이전트
루비


# 12장 지리정보기반 시각화: 지리정보와 시각화

## 12.2 포리움(Folium)의 설치 및 활용

In [44]:
import folium
map_osm = folium.Map(location=[37.566345, 126.977893])
map_osm.save('./map1.html')
map_osm = folium.Map(location=[37.566345, 126.977893], zoom_start=17)
map_osm.save('./map2.html')
map_osm = folium.Map(location=[37.566345, 126.977893], zoom_start=17, tiles='Stamen Terrain')
map_osm.save('./map3.html')
map_osm = folium.Map(location=[37.566345, 126.977893], zoom_start=17, tiles='Stamen Toner')
map_osm.save('./map4.html')